In [2]:
import pandas as pd
import networkx as nx
import osmnx as ox
import folium

In [3]:
# 경상남도 소방용수 현황 CSV 불러오기
import pandas as pd
fire_water = pd.read_csv('C:/Users/tmd46/Desktop/경상남도_경남소방본부_소방용수시설_20220614.csv', encoding='cp949')
fire_water.head()

,시설번호,시설유형코드,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,상세위치,안전센터명,보호틀유무,사용가능여부,설치연도,배관깊이,출수압력,배관지름,관할기관명,관할기관전화번호,데이터기준일자
0,상대-상-01,1,경상남도,진주시,48170,진주시 남강로 881번길 9,진주시 상대동 730-53,35.180839,128.101546,"구,법원뒷편 후문 80m",상대119안전센터,N,Y,1979,0.5,3.5,65,진주소방서,055-760-9296,2022-06-14
1,상대-상-02,1,경상남도,진주시,48170,진주시 동진로 105,진주시 상대동 295-8,35.179397,128.102865,디럭스빌딩(한보프라자) 주차장 입구,상대119안전센터,Y,Y,1979,0.5,3.5,65,진주소방서,055-760-9296,2022-06-14
2,상대-상-03,1,경상남도,진주시,48170,진주시 동진로 121,진주시 상대동 291-23,35.179447,128.104407,가나신협앞 인도상,상대119안전센터,Y,Y,1979,0.5,3.5,65,진주소방서,055-760-9296,2022-06-14
3,상대-상-05,1,경상남도,진주시,48170,진주시 동진로 155,진주시 상대동 284,35.179702,128.107293,진주시청 남서편 인도상 (진주시청 사거리),상대119안전센터,Y,Y,1979,0.5,3.5,65,진주소방서,055-760-9296,2022-06-14
4,상대-상-06,1,경상남도,진주시,48170,진주시 솔밭로 141,진주시 상대동 288-11,35.181777,128.106752,모란아파트 앞 인도상,상대119안전센터,Y,Y,1979,0.5,3.5,65,진주소방서,055-760-9296,2022-06-14


In [4]:
fire_water_jinju = fire_water.loc[fire_water["시군구명"] == "진주시"]
fire_water_jinju = fire_water_jinju.loc[:,["시군구명","위도","경도","상세위치","보호틀유무","사용가능여부","설치연도","배관깊이","출수압력","배관지름"]]

latitude = list(fire_water_jinju["위도"])     # 소방용수 위도 위치
longitude = list(fire_water_jinju["경도"])    # 소방용수 경도 위치

location = list(zip(latitude, longitude))    # 위도, 경도 데이터를 튜플로 묶어서 하나의 리스트에 담는다

In [6]:
lat, lon = map(float, input("화재발생지역의 위도와 경도를 입력해주세요 : ").split(','))   # 시뮬레이션 화재발생지역 위치지정
location_point = (lat, lon)  # 화재지역을 기준으로 한 지도 생성좌표
G1_1 = ox.graph_from_point(location_point, dist=10000, dist_type='network')   # 화재지역을 기준으로 한 지도 생성

orig_node = ox.nearest_nodes(G1_1,lon, lat)  # 화재 발생 위치(노드)
ten_km = []  # 0.6km 이내 소방용수 노드정보 저장
loca = []    # 0.6km 이내 소방용수 (위도, 경도) 정보 저장
info_detail = []    # 소방용수 상세위치 정보
info_protect = []   # 소방용수 보호틀 유무 정보
info_use = []   # 소방용수 사용가능 여부 정보
info_year = []  #  설치연도 정보
info_depth = []   # 배관깊이 정보
info_pressure = []  # 출수압력 정보
info_diameter = []   # 배관지름 정보
cnt = 0
index = 0
for x,y in location:
    dest_node = ox.nearest_nodes(G1_1,y,x)  # 소방용수 노드 (모든 소방용수 위치)
    len=nx.shortest_path_length(G1_1,orig_node,dest_node,weight='length')/1000    # 화재지역으로부터 거리(km)
    if (len < 0.6):        # 0.6km 이내 소방용수 검색
        ten_km.append(dest_node)   # 정보 추가
        cnt += 1     # 0.6km 이내 소방용수 개수
        loca.append((x,y))     # 정보 추가

        if (fire_water_jinju["위도"][index] == x and fire_water_jinju["경도"][index] == y):
            info_detail.append(fire_water_jinju["상세위치"][index])
            info_protect.append(fire_water_jinju["보호틀유무"][index])
            info_use.append(fire_water_jinju["사용가능여부"][index])
            info_year.append(fire_water_jinju["설치연도"][index])
            info_depth.append(fire_water_jinju["배관깊이"][index])
            info_pressure.append(fire_water_jinju["출수압력"][index])
            info_diameter.append(fire_water_jinju["배관지름"][index])
    index += 1

# 지도를 통해 조건에 맞는 소방용수 위치 띄우기
m = folium.Map([lat, lon], zoom_start = 15) 

folium.Circle(
    location=[lat, lon],
    radius=550, # 원 크기
    color='blue', # 원 선 색상
    fill_color='blue', # 원 내부 색상
).add_to(m)

for i in range(cnt):
    folium.Circle(
        location = loca[i],
        radius = 10,
        color = 'blue',
        fill = 'crimson',
        tooltip=f"""상세위치는 {info_detail[i]}입니다.<br/>보호틀유무 : 
        {info_protect[i]}<br/>사용가능여부 : 
        {info_use[i]}<br/>설치연도 : {info_year[i]}<br/>배관깊이 : {info_depth[i]}<br/>출수압력 : 
        {info_pressure[i]}<br/>배관지름 : {info_diameter[i]}"""
    ).add_to(m)

folium.Marker(
    location=[lat, lon],
    icon = folium.Icon(color='red', icon='star') # 불난 곳 표시
).add_to(m)

print(f"반경 600m 이내에 존재하는 소방용수의 개수는 {cnt}개 입니다.")
m

화재발생지역의 위도와 경도를 입력해주세요 : 35.189882699999934,128.07932599999972
반경 600m 이내에 존재하는 소방용수의 개수는 8개 입니다.
